In [1]:
owner = 'test-18'

In [2]:
import polaris as po
import datamol as dm
import numpy as np

/home/ubuntu/micromamba/envs/moml-ilya/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [51]:
## Balancing multi-task

from functools import partial

from imblearn.over_sampling import SMOTE
import polaris as po
import datamol as dm
import numpy as np

mapping = {
 (0.0, 0.0, 0.0): 0,
 (1.0, 0.0, 0.0): 1,
 (1.0, 0.0, 1.0): 2,
 (1.0, 1.0, 0.0): 3,
 (1.0, 1.0, 1.0): 4,
}
inv_mapping = {v: k for k, v in mapping.items()}

# load dataset
benchmark = po.load_benchmark("polaris/pkis1-kit-wt-mut-c-1")
# use ECFP fingerprint
train, test = benchmark.get_train_test_split(featurization_fn=partial(dm.to_fp, fp_type='ecfp'))

# define order of target values
target_order = ['CLASS_KIT', 'CLASS_KIT_(T6701_mutant)', 'CLASS_KIT_(V560G_mutant)']

# reshape the y values for convenience
ys = train.y
ys = np.stack([ys[target] for target in target_order], axis=1)
ys.shape

# remove the rows with NaN values
mask = ~np.any(np.isnan(ys), axis=1)
mask.sum()
X = train.X[mask]
ys = ys[mask]

ys_scalarized = [tuple(item) for item in ys]
ys_scalarized = [mapping[item] for i, item in enumerate(ys_scalarized)]

X_resampled, y_resampled = SMOTE(k_neighbors=2).fit_resample(X, ys_scalarized)
y_resampled = [inv_mapping[item] for i, item in enumerate(y_resampled)]


X_train = X_resampled
y_train = y_resampled

2024-06-21 17:45:44.314 | INFO     | polaris._artifact:_validate_version:66 - The version of Polaris that was used to create the artifact (0.0.0) is different from the currently installed version of Polaris (dev).
2024-06-21 17:45:44.318 | INFO     | polaris._artifact:_validate_version:66 - The version of Polaris that was used to create the artifact (0.0.0) is different from the currently installed version of Polaris (dev).


In [28]:
from sklearn.cluster import KMeans
from sklearn.manifold import TSNE
from sklearn.model_selection import StratifiedKFold
from sklearn.linear_model import LogisticRegression
# Perform clustering using KMeans
num_clusters = 5  # Adjust if needed
kmeans = KMeans(n_clusters=num_clusters, random_state=42)
cluster_labels = kmeans.fit_predict(X_train)

# --- Analyze and Visualize Clusters ---
print("Number of clusters:", num_clusters)
print("Cluster sizes:", [np.sum(cluster_labels == i) for i in range(num_clusters)])

# Perform t-SNE for dimensionality reduction
tsne = TSNE(n_components=2, random_state=42)  
embeddings = tsne.fit_transform(X_train)

# # Plot the clusters
# plt.figure(figsize=(8, 6))
# for cluster_id in range(num_clusters):
#     plt.scatter(embeddings[cluster_labels == cluster_id, 0], 
#                 embeddings[cluster_labels == cluster_id, 1],
#                 label=f"Cluster {cluster_id + 1}")

# plt.xlabel("t-SNE Dimension 1")
# plt.ylabel("t-SNE Dimension 2")
# plt.title("t-SNE Visualization of Molecule Clusters")
# plt.legend()
# plt.show()

# --- Cross-Validation ---

# 5-fold cross-validation over clusters
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
for fold, (train_index, val_index) in enumerate(skf.split(X_train, cluster_labels)):
    print(f"Fold {fold + 1}:")
    print(f"  TRAIN: {len(train_index)} ligands")
    print(f"  VALIDATION: {len(val_index)} ligands")

Number of clusters: 5
Cluster sizes: [14, 81, 1037, 35, 63]
Fold 1:
  TRAIN: 984 ligands
  VALIDATION: 246 ligands
Fold 2:
  TRAIN: 984 ligands
  VALIDATION: 246 ligands
Fold 3:
  TRAIN: 984 ligands
  VALIDATION: 246 ligands
Fold 4:
  TRAIN: 984 ligands
  VALIDATION: 246 ligands
Fold 5:
  TRAIN: 984 ligands
  VALIDATION: 246 ligands


In [5]:
import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split, cross_val_score
from bayes_opt import BayesianOptimization


def get_param(optimizer):
    best_params = optimizer.max['params']
    best_params['input_dim'] = int(best_params['input_dim'])
    best_params['n_estimators'] = int(best_params['n_estimators'])
    best_params['max_depth'] = int(best_params['max_depth'])
    best_params['min_samples_split'] = int(best_params['min_samples_split'])
    best_params['min_samples_leaf'] = int(best_params['min_samples_leaf'])
    best_params['bootstrap'] = bool(best_params['bootstrap'])
    return best_params

In [31]:
import numpy as np
from sklearn.model_selection import KFold, cross_val_score, train_test_split
from bayes_opt import BayesianOptimization
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.metrics import make_scorer, average_precision_score
from sklearn.ensemble import RandomForestClassifier


def opt_multi_targets_pca(param_bounds):

    X_train = X_resampled
    y_train = y_resampled
    # X_train = train.X[mask]
    # y_train = ys[mask]

    def objective(input_dim, n_estimators, max_depth, min_samples_split, min_samples_leaf, max_features, bootstrap):
        n_estimators = int(n_estimators)
        max_depth = int(max_depth)
        min_samples_split = int(min_samples_split)
        min_samples_leaf = int(min_samples_leaf)
        max_features = max(min(max_features, 0.999), 1e-3)  # to avoid 0 and 1
        
        rf = RandomForestClassifier(
            n_estimators=n_estimators,
            max_depth=max_depth,
            min_samples_split=min_samples_split,
            min_samples_leaf=min_samples_leaf,
            max_features=max_features,
            bootstrap=bool(bootstrap),
            random_state=42
        )

        pipeline = Pipeline([
            ('pca', PCA(int(input_dim))),
            ('scaler', StandardScaler()),
            ('rf', rf)
        ])
        
        # Use K-Fold cross-validation
        #kfold = KFold(n_splits=2, shuffle=True, random_state=42)

        accuracy = 0      
        for target_label in target_order:
                print(f"Cross-Validation for Target: {target_label}")
                skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
            
                # Get the index of the target label
                target_index = target_order.index(target_label)
                #scoring = make_scorer(average_precision_score, needs_proba=True)
                # Correctly call cross_val_score (no inner loop)
                scores = cross_val_score(pipeline, X_train, [sample[target_index] for sample in y_train], cv=skf, scoring="average_precision")  
            
                accuracy += scores.mean()
            
        return accuracy / len(target_order) # Average accuracy across all targets
        # return accuracy / len(target_order) # Average accuracy across all targets
        
        
        # # Evaluate the model using cross-validation
        # scoring = make_scorer(average_precision_score, needs_proba=True)
        
        # cv_scores = cross_val_score(pipeline, X_train, y_train, cv=kfold, scoring="average_precision")
        
    
    optimizer = BayesianOptimization(
        f=objective,
        pbounds=param_bounds,
        random_state=42,
        verbose=2
    )
    
    optimizer.maximize(init_points=10, n_iter=50)

        
    best_params = get_param(optimizer)
    
    print("Best parameters found: ", best_params)
    
    return best_params


In [32]:
param_bounds = {
    'n_estimators': (10, 200),
    'max_depth': (1, 50),
    'input_dim': (10, 100),
    'min_samples_split': (2, 10),
    'min_samples_leaf': (1, 10),
    'max_features': (0.1, 0.999),
    'bootstrap': (0, 1)  # Treated as a boolean
}

best_params = opt_multi_targets_pca(param_bounds = param_bounds)


|   iter    |  target   | bootstrap | input_dim | max_depth | max_fe... | min_sa... | min_sa... | n_esti... |
-------------------------------------------------------------------------------------------------------------
Cross-Validation for Target: CLASS_KIT
Cross-Validation for Target: CLASS_KIT_(T6701_mutant)
Cross-Validation for Target: CLASS_KIT_(V560G_mutant)
| 1         | 0.9998    | 0.3745    | 95.56     | 36.87     | 0.6382    | 2.404     | 3.248     | 21.04     |
Cross-Validation for Target: CLASS_KIT
Cross-Validation for Target: CLASS_KIT_(T6701_mutant)
Cross-Validation for Target: CLASS_KIT_(V560G_mutant)
| 2         | 0.9991    | 0.8662    | 64.1      | 35.7      | 0.1185    | 9.729     | 8.66      | 50.34     |
Cross-Validation for Target: CLASS_KIT
Cross-Validation for Target: CLASS_KIT_(T6701_mutant)
Cross-Validation for Target: CLASS_KIT_(V560G_mutant)
| 3         | 0.9991    | 0.1818    | 26.51     | 15.91     | 0.5718    | 4.888     | 4.33      | 126.3     |
Cross-Val

In [33]:
best_rf = RandomForestClassifier(
        n_estimators=best_params['n_estimators'],
        max_depth=best_params['max_depth'],
        min_samples_split=best_params['min_samples_split'],
        min_samples_leaf=best_params['min_samples_leaf'],
        max_features=best_params['max_features'],
        bootstrap=best_params['bootstrap'],
        random_state=42
    )

pca = PCA(n_components = int(best_params['input_dim']))

X_train_pca = pca.fit_transform(X_train)
X_test_pca = pca.transform(test.X)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_pca)
X_test_scaled = scaler.transform(X_test_pca)

best_rf.fit(X_train_scaled, y_train)


y_pred = best_rf.predict(X_test_scaled)
#print(y_pred)

y_prob = best_rf.predict_proba(X_test_scaled)
y_prob = np.stack(y_prob, axis=1)
#print(y_prob)

y_pred_multi = {k: y_pred[:, idx] for idx, k in enumerate(benchmark.target_cols)}
y_prob_multi = {k: y_prob[:, idx, 1] for idx, k in enumerate(benchmark.target_cols)}




results_multi = benchmark.evaluate(y_pred=y_pred_multi, y_prob=y_prob_multi)


results_multi.name = "rf_multi_augm_pca_clusters"
results_multi.description = best_params
results_multi.to_json('rf_multi_augm_pca_clusters.json')




/home/ubuntu/micromamba/envs/moml-ilya/lib/python3.11/site-packages/pydantic/main.py:347: UserWarning: Pydantic serializer warnings:
  Expected `str` but got `dict` - serialized value may not be as expected
  return self.__pydantic_serializer__.to_python(


In [74]:
import numpy as np
from sklearn.model_selection import KFold
from sklearn.base import clone


#print(y_train)
# List of models
models = [best_rf, best_rf, best_rf, best_rf, best_rf]

# Initialize KFold
kf = KFold(n_splits=5, shuffle=True, random_state=42)
#kf = KFold(n_splits=5, shuffle=True, random_state=42)



y_train = np.array(y_train)
#pca = PCA(n_components = int(best_params['input_dim']))


#X_test_pca = pca.transform(test.X)
#X_test_scaled = scaler.transform(X_test_pca)

# Dictionary to store models from each fold
cv_models = {i: [] for i in range(kf.get_n_splits())}

# Perform cross-validation manually
for fold_idx, (train_idx, val_idx) in enumerate(kf.split(X_train)):
    print(type(train_idx))
    #print(y_train)
    X_train_1, X_val = X_train[train_idx], X_train[val_idx]
    y_train_1, y_val = y_train[train_idx], y_train[val_idx]
    
    pca = PCA(n_components = int(best_params['input_dim']))
    
    X_train_pca_1 = pca.fit_transform(X_train_1)
   
    
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train_pca_1)

    
    for model in models:
        # Clone the model to ensure each fold gets a fresh model
        cloned_model = clone(model)
        cloned_model.fit(X_train_scaled, y_train_1)
        
        # Save the trained model
        cv_models[fold_idx].append(cloned_model)



<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>


In [75]:
from sklearn.ensemble import VotingClassifier

# Create a list of (name, model) tuples
estimators = [('model1', cv_models[0]), ('model2', cv_models[1]), ('model3', cv_models[2]), ('model4', cv_models[3]), ('model5', cv_models[4])]

# Initialize VotingClassifier
voting_clf = VotingClassifier(estimators=estimators, voting='hard')  # 'hard' for majority voting

pca = PCA(n_components = int(best_params['input_dim']))

X_train_pca = pca.fit_transform(X_train)
X_test_pca = pca.transform(test.X)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_pca)
X_test_scaled = scaler.transform(X_test_pca)

# Fit and predict
voting_clf.fit(X_train_scaled, y_train)
final_predictions = voting_clf.predict(X_test)




NotImplementedError: VotingClassifier only supports binary or multiclass classification. Multilabel and multi-output classification are not supported.

In [50]:
results = results_multi

# Please use the `ML4DD-team#` template for the name
results.name = "ML4DD-team18"

# Short description of your method
results.description = "Kinase inhibitors with a Random Forest by a bunch of amazing people"

# A link to your code, e.g. Github or Google Colab.
results.github_url = "https://github.com/markf94/moml-2024-hackathon"

# A link to a short (<1 page) write-up of your method 
# e.g. in Google Docs or Notion
results.paper_url = "https://docs.google.com/document/d/14PNqLiuyHmuNNVLyNNSQpAlnBdmKc1TqhpBqzdRy5qI/edit?usp=sharing"

# Specify the username of all your team members
results.contributors = ["qumark"]


results.upload_to_hub(owner='viprorok')

2024-06-21 17:44:58.802 | SUCCESS  | polaris.hub.client:upload_results:492 - Your result has been successfully uploaded to the Hub. View it here: https://polarishub.io/benchmarks/polaris/pkis1-kit-wt-mut-c-1/s3eB9wxNJBFqvUyp89ajD


{'id': 's3eB9wxNJBFqvUyp89ajD',
 'createdAt': '2024-06-21T17:44:58.759Z',
 'deletedAt': None,
 'name': 'ML4DD-team18',
 'slug': 'ml4dd-team18',
 'description': 'Kinase inhibitors with a Random Forest by a bunch of amazing people',
 'tags': [],
 'userAttributes': {},
 'access': 'private',
 'isCertified': False,
 'polarisVersion': 'dev',
 'ownerId': 'Rr0SvnCcKhrc6vP2ahPhe',
 'creatorId': 'Rr0SvnCcKhrc6vP2ahPhe',
 'benchmarkId': 'DZzlykxvBwlSA9uERL17A',
 'results': [{'scores': {'f1': 0,
    'mcc': 0,
    'pr_auc': 0.697615598235066,
    'roc_auc': 0.7477553310886645,
    'accuracy': 0.6206896551724138,
    'cohen_kappa': 0},
   'testSet': 'test',
   'targetLabel': 'CLASS_KIT'},
  {'scores': {'f1': 0.25,
    'mcc': 0.3502700038973536,
    'pr_auc': 0.6723174745342232,
    'roc_auc': 0.8620352250489237,
    'accuracy': 0.8620689655172413,
    'cohen_kappa': 0.21856287425149712},
   'testSet': 'test',
   'targetLabel': 'CLASS_KIT_(T6701_mutant)'},
  {'scores': {'f1': 0,
    'mcc': 0,
    'pr

In [150]:
def opt_target(target, param_bounds):
    if isinstance(target, int) is True:
        target = benchmark.target_cols[target]
    
    ys = train.y[target]
    y_train = ys[mask]
    
    def rf_cv(n_estimators, max_depth, min_samples_split, min_samples_leaf, max_features, bootstrap):
        # Convert parameters to int where necessary
        n_estimators = int(n_estimators)
        max_depth = int(max_depth)
        min_samples_split = int(min_samples_split)
        min_samples_leaf = int(min_samples_leaf)
        max_features = max(min(max_features, 0.999), 1e-3)  # to avoid 0 and 1
    
        rf = RandomForestClassifier(
            n_estimators=n_estimators,
            max_depth=max_depth,
            min_samples_split=min_samples_split,
            min_samples_leaf=min_samples_leaf,
            max_features=max_features,
            bootstrap=bool(bootstrap),
            random_state=42
        )
        
        # Perform cross-validation
        cval = cross_val_score(rf, X_train, y_train, scoring='accuracy', cv=3)
        
        return cval.mean()
    
    
    
    optimizer = BayesianOptimization(
        f=rf_cv,
        pbounds=param_bounds,
        random_state=42,
        verbose=2
    )
    optimizer.maximize(init_points=10, n_iter=30)
    best_params = get_param(optimizer)

    best_rf = RandomForestClassifier(
        n_estimators=best_params['n_estimators'],
        max_depth=best_params['max_depth'],
        min_samples_split=best_params['min_samples_split'],
        min_samples_leaf=best_params['min_samples_leaf'],
        max_features=best_params['max_features'],
        bootstrap=best_params['bootstrap'],
        random_state=42
    )
    
    best_rf.fit(X_train, y_train)

    return best_rf, best_params


In [151]:
param_bounds = {
    'n_estimators': (10, 200),
    'max_depth': (1, 50),
    'min_samples_split': (2, 10),
    'min_samples_leaf': (1, 10),
    'max_features': (0.1, 0.999),
    'bootstrap': (0, 1)  # Treated as a boolean
}

models = {target: opt_target(target, param_bounds = param_bounds) for target in benchmark.target_cols}

|   iter    |  target   | bootstrap | max_depth | max_fe... | min_sa... | min_sa... | n_esti... |
-------------------------------------------------------------------------------------------------
| 1         | 0.9746    | 0.3745    | 47.59     | 0.7581    | 6.388     | 3.248     | 39.64     |
| 2         | 0.9746    | 0.05808   | 43.44     | 0.6404    | 7.373     | 2.165     | 194.3     |
| 3         | 0.9746    | 0.8324    | 11.4      | 0.2635    | 2.651     | 4.434     | 109.7     |
| 4         | 0.9746    | 0.4319    | 15.27     | 0.6501    | 2.255     | 4.337     | 79.61     |
| 5         | 0.9746    | 0.4561    | 39.47     | 0.2795    | 5.628     | 6.739     | 18.83     |
| 6         | 0.9746    | 0.6075    | 9.356     | 0.1585    | 9.54      | 9.725     | 163.6     |
| 7         | 0.9746    | 0.3046    | 5.786     | 0.7151    | 4.961     | 2.976     | 104.1     |
| 8         | 0.9746    | 0.03439   | 45.56     | 0.3326    | 6.963     | 4.494     | 108.8     |
| 9         | 0.9746

In [158]:
y_prob_ind = {target: model.predict_proba(test.X)[:, 1] for target, (model, params) in models.items()}
y_pred_ind = {target: model.predict(test.X) for target, (model, params) in models.items()}

best_params_ind = {target: params for target, (model, params) in models.items()}

results_ind = benchmark.evaluate(y_pred=y_pred_ind, y_prob=y_prob_ind)

results_ind.name = "rf_ind"
results_ind.description = best_params_ind
results_ind.to_json('rf_ind.json')

/home/ubuntu/micromamba/envs/moml-ilya/lib/python3.11/site-packages/pydantic/main.py:347: UserWarning: Pydantic serializer warnings:
  Expected `str` but got `dict` - serialized value may not be as expected
  return self.__pydantic_serializer__.to_python(


In [159]:
results_ind

{
  "name": "rf_ind",
  "description": {
    "CLASS_KIT_(T6701_mutant)": {
      "bootstrap": true,
      "max_depth": 47,
      "max_features": 0.7580625536884532,
      "min_samples_leaf": 6,
      "min_samples_split": 3,
      "n_estimators": 39
    },
    "CLASS_KIT_(V560G_mutant)": {
      "bootstrap": true,
      "max_depth": 47,
      "max_features": 0.7580625536884532,
      "min_samples_leaf": 6,
      "min_samples_split": 3,
      "n_estimators": 39
    },
    "CLASS_KIT": {
      "bootstrap": true,
      "max_depth": 35,
      "max_features": 0.7658783536012476,
      "min_samples_leaf": 2,
      "min_samples_split": 7,
      "n_estimators": 178
    }
  },
  "tags": [],
  "user_attributes": {},
  "owner": null,
  "polaris_version": "dev",
  "benchmark_name": "pkis1-kit-wt-mut-c-1",
  "benchmark_owner": {
    "slug": "polaris",
    "external_id": "org_2gtoaJIVrgRqiIR8Qm5BnpFCbxu",
    "type": "organization"
  },
  "github_url": null,
  "paper_url": null,
  "contributors": null,
  "artifact_id": null,
  "benchmark_artifact_id": "polaris/pkis1-kit-wt-mut-c-1",
  "results": [
    {
      "Test set": "test",
      "Target label": "CLASS_KIT_(T6701_mutant)",
      "Metric": "accuracy",
      "Score": 0.8390804598
    },
    {
      "Test set": "test",
      "Target label": "CLASS_KIT_(V560G_mutant)",
      "Metric": "accuracy",
      "Score": 0.8620689655
    },
    {
      "Test set": "test",
      "Target label": "CLASS_KIT",
      "Metric": "accuracy",
      "Score": 0.6551724138
    },
    {
      "Test set": "test",
      "Target label": "CLASS_KIT_(T6701_mutant)",
      "Metric": "f1",
      "Score": 0.0
    },
    {
      "Test set": "test",
      "Target label": "CLASS_KIT_(V560G_mutant)",
      "Metric": "f1",
      "Score": 0.0
    },
    {
      "Test set": "test",
      "Target label": "CLASS_KIT",
      "Metric": "f1",
      "Score": 0.2105263158
    },
    {
      "Test set": "test",
      "Target label": "CLASS_KIT_(T6701_mutant)",
      "Metric": "roc_auc",
      "Score": 0.7353228963
    },
    {
      "Test set": "test",
      "Target label": "CLASS_KIT_(V560G_mutant)",
      "Metric": "roc_auc",
      "Score": 0.7227777778
    },
    {
      "Test set": "test",
      "Target label": "CLASS_KIT",
      "Metric": "roc_auc",
      "Score": 0.8010662177
    },
    {
      "Test set": "test",
      "Target label": "CLASS_KIT_(T6701_mutant)",
      "Metric": "pr_auc",
      "Score": 0.6131561014
    },
    {
      "Test set": "test",
      "Target label": "CLASS_KIT_(V560G_mutant)",
      "Metric": "pr_auc",
      "Score": 0.4876573618
    },
    {
      "Test set": "test",
      "Target label": "CLASS_KIT",
      "Metric": "pr_auc",
      "Score": 0.6506629159
    },
    {
      "Test set": "test",
      "Target label": "CLASS_KIT_(T6701_mutant)",
      "Metric": "mcc",
      "Score": 0.0
    },
    {
      "Test set": "test",
      "Target label": "CLASS_KIT_(V560G_mutant)",
      "Metric": "mcc",
      "Score": 0.0
    },
    {
      "Test set": "test",
      "Target label": "CLASS_KIT",
      "Metric": "mcc",
      "Score": 0.2140944019
    },
    {
      "Test set": "test",
      "Target label": "CLASS_KIT_(T6701_mutant)",
      "Metric": "cohen_kappa",
      "Score": 0.0
    },
    {
      "Test set": "test",
      "Target label": "CLASS_KIT_(V560G_mutant)",
      "Metric": "cohen_kappa",
      "Score": 0.0
    },
    {
      "Test set": "test",
      "Target label": "CLASS_KIT",
      "Metric": "cohen_kappa",
      "Score": 0.122983871
    }
  ]
}